# <span style="color: blue;">Магические методы</span>

"Магическими" называются внутренние методы классов, например, метод `__init__`.

С помощью "магических" методов можно:
* управлять доступом к атрибутам экземпляра
* перегрузить операторы (сравнения, арифметические, и др.)
* определить строковое представление экземпляра
* изменить способ хеширования экземпляра
* и др.

Мы рассмотрим только часть наиболее используемых методов.

Подробное описание всех "магических" методов можно найти в документации языка: http://bit.ly/magic-methods

### Метод `__getattr__`

Метод __getattr__ вызывается при попытке прочитать значение несуществующего атрибута:

In [ ]:
class Noop:
    pass

Noop().foobar

Определим метод `__getattr__` для класса `Noop`:

In [ ]:
class Noop:
    def __getattr__(self, name):
        return name  # identity
    
Noop().foobar

### Методы `__setattr__`и `__detattr__`

Методы `__setattr__` и `_delattr_` позволяют управлять изменением значения и удалением атрибутов.

В отличие от `__getattr__` они вызываются для всех атрибутов, а не только для несуществующих.

Пример, запрещающий изменять значение некоторых атрибутов:

In [ ]:
class Guarded:
    guarded = []
    
    def __setattr__(self, name, value):
        assert name not in self.guarded
        super().__setattr__(name, value)
        
class Noop(Guarded):
    guarded = ["foobar"]
    
    def __init__(self):
        self.__dict__["foobar"] = 42  # Зачем это?

In [ ]:
a = Noop()
a.foobar = 1

In [ ]:
a.__dict__["foobar"] = 2
a.__dict__["foobar"]

In [ ]:
a.another = 1
a.another

### Функции __getattr__, __setattr__, __delattr__ и __getattribute__

Функция `getattr` позволяет безопасно получить значение атрибута экземпляра класса по его имени:

In [ ]:
class Noop:
    some_attribute = 42

In [ ]:
noop = Noop()
getattr(noop, "some_attribute")

In [ ]:
getattr(noop, "some_other_attribute", 100500)

Комплементарные функции `setattr` и `delattr` добавляют и удаляют атрибут:

In [ ]:
setattr(noop, "some_other_attribute", 100500)
delattr(noop, "some_other_attribute")

`__getattribute__` вызывается **до** обращения к атрибуту<br/>
_(написать пример?)_

In [ ]:
from collections import Counter

class CountedMixin:
    def __init__(self):
        self.__counter = Counter()
        
    def __getattribute__(self, name):
        attr = object.__getattribute__(self, name)
        if attr and callable(attr):
            self.__counter[name] += 1
            print("Attribute {} called {} times".format(name, self.__counter[name]))
        return attr
    
class Greeter:
    def greet(self):
        print('Hi')
        
class CountedGreeter(CountedMixin, Greeter):
    pass

c = CountedGreeter()
c.greet()

In [ ]:
c.greet()

### Операторы сравнения

Чтобы экземпляры класса поддерживали все операторы сравнения, нужно реализовать внушительное количество "магических" методов:

In [ ]:
instance.__eq__(other)  # instance == other
instance.__ne__(other)  # instance != other
instance.__lt__(other)  # instance < other
instance.__le__(other)  # instance <= other
instance.__gt__(other)  # instance > other
instance.__ge__(other)  # instance >= other

С помощью специального декоратора можно облегчить реализацию операторов сравнения:

In [ ]:
import functools

@functools.total_ordering
class Counter:
    def __eq__(self, other):
        return self.value == other.value
    
    def __lt__(self, other):  # или <=, >, >=
        return self.value < other.value

### Метод __call__

Метод `__call__` позволяет "вызывать" экземпляры классов, имитируя интерфейс функций:

In [ ]:
class Identity:
    def __call__(self, x):
        return x
    
Identity()(42)

### Преобразование в строку

In [ ]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
        
    def __repr__(self):
        return "Counter({})".format(self.value)
    
    def __str__(self):
        return "Counted to {}".format(self.value)
    
c = Counter(42)
c

In [ ]:
print(c)

### Расширение __format__

In [ ]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
    
    def __format__(self, format_spec):
        return self.value.__format__(format_spec)

c = Counter(42)
"Counted to {:b}".format(c)

In [1]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
    
    def __format__(self, format_spec):
        if '@' in format_spec:
            format_spec = format_spec.replace('@', '')
            default = self.value.__format__(format_spec)
            return "[hello:{}]".format(default)
        else:
            return self.value.__format__(format_spec)

c = Counter(42)
"Counted to {:b@}".format(c)

'Counted to [hello:101010]'

### Метод __hash__

Метод `__hash__` используется для вычисления значения хеш-функции

Реализация по умолчанию гарантирует, что одинаковое значение хеш функции будет только у физически одинаковых объектов, то есть:

In [ ]:
x is y  <=>  hash(x) == hash(y)

Несколько очевидных рекомендаций:
* Метод `__hash__` имеет смысл реализовать только вместе с `__eq__`.<br/>При этом нужно соблюсти: `x == y  =>  hash(x) == hash(y)`
* Для изменяемых объектов можно ограничиться только методом `__eq__`

### Метод __bool__

Метод `__bool__` для проверки значения на истинность, например, в условии оператора `if`

Для класса `Counter` реализация `__bool__` тривиальна:

In [ ]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
        
    def __bool__(self):
        return bool(self.value)
    
c = Counter()
if not c:
    print("No counts yet.")

### Ещё:

* **арифметические**: `add`, `radd`, `iadd`, `mul`, `div`, ...
* **сравнения**: `eq`, `neq`, `le`, `lt`, `gt`, `ge`
* **преобразования типов**: `int`, `str`, `bool`, ...
* **управление отображением в интерпретаторе**: `repr`
* **доступ к элементам**: `getitem`, `setitem`, `len`, ...
* **доступ к атрибутам**: `getattr`, `setattr`, `delattr`, `getattribute`
* **хеширования**: `hash`
* **вызова**: `call`

### Резюме

"Магические" методы позволяют уточнить поведение экземпляров класса в различных конструкциях языка.

Мы рассмотрели небольшое количество "магических" методов, на самом деле их намного больше.